In [17]:
import os
import csv
import subprocess
import array
import random
import pandas as pd
import numpy as np
from pyAudioAnalysis import audioSegmentation as aS
from pyAudioAnalysis import audioTrainTest as aT
from itertools import groupby
from operator import itemgetter
from IPython.display import display, Audio
from pydub import AudioSegment
from pydub.utils import get_array_type
import matplotlib.pyplot as plt
import shutil
import timeit
%matplotlib inline

os.chdir('/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips')

In [18]:
speaker="King, Martin Luther"

last_name = speaker.split(', ')[0]

print(speaker)
print(last_name)

King, Martin Luther
King


In [19]:
## Loading table of labeled 2-second audio segments

aapb_metadata_all_split=pd.read_csv("/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/AAPB_ARLO_All_170414_2_sec_segs.csv")
aapb_metadata_all_split.tail(1)

,AAPB Unique Identifier (GUID),Type,Value,class ID,Timecode IN,Timecode OUT,Tag Duration,File Duration,SonyCi ID,Filename,Pathname
178273,cpb-aacip/15-9w37kv75,Sound quality,audience clapping,NaN,223,225,2,1319.976,090d1475c56c45808fad6567b2f88b3d,cpb-aacip-15-9w37kv75__213877_,/Volumes/McLaughlin-6TB-1/Extended_Corpus/Gera...


In [20]:
## Function for extracting speaker clips

def extract_clips(x_table):
    for unique_audio_path in sorted(list(set(list(x_table['Pathname'])))):
        inputfile=unique_audio_path
        if os.path.exists(inputfile):
            if inputfile.lower()[-4:].lower() in ('.wav','.mp3','.mp4'):
                wav_source=True
                if inputfile.lower()[-4:]=='.mp4':     # Creates a temporary WAV
                    wav_source=False                         # if input is MP4
                    temp_filename=inputfile.split('/')[-1]+'_temp.wav'
                    audio_path='/var/tmp/'+temp_filename   # Pathname for temp WAV
                    subprocess.call(['ffmpeg', '-y', '-i', inputfile, audio_path]) # '-y' option overwrites existing file if present
                else:
                    audio_path=inputfile

                song=None
                from pydub import AudioSegment
                
                if inputfile[-4:].lower()=='.mp3':
                    song = AudioSegment.from_mp3(audio_path)
                else:
                    song = AudioSegment.from_wav(audio_path)


        for index, rowref in x_table[x_table['Pathname']==unique_audio_path].iterrows():
            row=aapb_metadata_all_split.iloc[index] ## Ref to master split table
            time_in=row['Timecode IN']
            clip_duration=row['Tag Duration']
            inputfile=row['Pathname']
            basename=row['Filename']
            dir_name=row['Value']
            
            try: os.mkdir(dir_name)
            except: pass

        
            from pydub import AudioSegment 
            start_msec = float(time_in) * 1000.0
            duration_msec = float(clip_duration) * 1000
            clip_pathname=basename+'.start_'+str(time_in)[:8]+'.dur_2s.wav'
            if not os.path.exists(os.path.join(dir_name,clip_pathname)):
                clip_data = song[start_msec:start_msec+duration_msec]
                clip_data=clip_data.set_channels(1)
                clip_data.export(os.path.join(dir_name,clip_pathname), format="wav")

        try:
            if wav_source==False:
                os.remove(audio_path)
        except: pass

    print("*** All segments extracted! ***")



In [5]:
print(str(len(aapb_metadata_all_split[aapb_metadata_all_split['Value']==speaker]))+" clips available for "+speaker)

21179 clips available for King, Martin Luther


In [21]:
##### Running the Random Clip Extractor #####


num_clips=4000     ## number of 2-second clips desired

x_table=aapb_metadata_all_split[aapb_metadata_all_split['Value']==speaker].sample(n=num_clips)

extract_clips(x_table)



*** All segments extracted! ***


In [22]:
## Creating ID string appended to each model's filename

clip_count=len(os.listdir(speaker))

model_id = '_'+last_name+'_UBM_'+str(clip_count)+'x2s'


In [23]:
## Assembling UBM files

os.chdir('/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips')

new_ubm_dir='UBM'+'_'+last_name


try: os.mkdir(new_ubm_dir)
except: pass

ubm_source="/Volumes/U/AAPB_Corpus_May_2017/test_set_616_clips"
ubm_files=[item for item in os.listdir(ubm_source) if (speaker not in item)&('16000.wav' in item)]
ubm_files=random.sample(ubm_files,60)
for filename in ubm_files:
    shutil.copy(os.path.join(ubm_source,filename),new_ubm_dir)

    
ubm_source="/Volumes/U/AAPB_Corpus_May_2017/PennSound_UBM_for_Creeley_full_clips_16000"
ubm_files=[item for item in os.listdir(ubm_source) if (speaker not in item)&('.wav' in item)]
ubm_files=random.sample(ubm_files,60)
for filename in ubm_files:
    shutil.copy(os.path.join(ubm_source,filename),new_ubm_dir)



In [24]:

## Train Model
import timeit
tic=timeit.default_timer()
#print(timeit.default_timer() - tic)



os.chdir('/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips')


aT.featureAndTrain([new_ubm_dir,speaker], 1.0, 1.0, aT.shortTermWindow, aT.shortTermStep, "svm", "svm"+model_id, False)
print("done")
print(timeit.default_timer() - tic)



Feature extraction complexity ratio: 60.9 x realtime
Feature extraction complexity ratio: 74.9 x realtime
Number of training experiments changed to 50 due to high number of samples
Number of training experiments changed to 10 due to high number of samples
Param = 0.00100 - Classifier Evaluation Experiment 1 of 10
Param = 0.00100 - Classifier Evaluation Experiment 2 of 10
Param = 0.00100 - Classifier Evaluation Experiment 3 of 10
Param = 0.00100 - Classifier Evaluation Experiment 4 of 10
Param = 0.00100 - Classifier Evaluation Experiment 5 of 10
Param = 0.00100 - Classifier Evaluation Experiment 6 of 10
Param = 0.00100 - Classifier Evaluation Experiment 7 of 10
Param = 0.00100 - Classifier Evaluation Experiment 8 of 10
Param = 0.00100 - Classifier Evaluation Experiment 9 of 10
Param = 0.00100 - Classifier Evaluation Experiment 10 of 10
Param = 0.01000 - Classifier Evaluation Experiment 1 of 10
Param = 0.01000 - Classifier Evaluation Experiment 2 of 10
Param = 0.01000 - Classifier Evalua

In [25]:
print(timeit.default_timer() - tic)/60.0

10.5452541868


In [ ]:
## Other classifiers

os.chdir('/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips')


tic=timeit.default_timer()
aT.featureAndTrain([new_ubm_dir,speaker], 1.0, 1.0, aT.shortTermWindow, aT.shortTermStep, "gradientboosting", "gradientboosting"+model_id, False)
print("done")
print(timeit.default_timer() - tic)
tic=timeit.default_timer()
aT.featureAndTrain([new_ubm_dir,speaker], 1.0, 1.0, aT.shortTermWindow, aT.shortTermStep, "extratrees", "extratrees"+model_id, False)
print("done")
print(timeit.default_timer() - tic)
tic=timeit.default_timer()
aT.featureAndTrain([new_ubm_dir,speaker], 1.0, 1.0, aT.shortTermWindow, aT.shortTermStep, "randomforest", "randomforest"+model_id, False)
print("done")
print(timeit.default_timer() - tic)
tic=timeit.default_timer()
aT.featureAndTrain([new_ubm_dir,speaker], 1.0, 1.0, aT.shortTermWindow, aT.shortTermStep, "knn", "knn"+model_id, False)
print("done")
print(timeit.default_timer() - tic)

In [ ]:
print(timeit.default_timer() - tic)/60.0

In [26]:
# Takes list of 1-second segments classified as speaker (1.0) or 
# non-speaker (0.0) and returns list of 2-tuples specifying speaker ranges.
# Add 1 to 2nd speaker in each 2-tuple for inclusive time span.
def seconds_list_to_ranges(seconds_list): 
    ranges = []                
    for k, g in groupby(enumerate(seconds_list), lambda (i,x):i-x):
        group = map(itemgetter(1), g)
        ranges.append((group[0], group[-1]))
    return ranges


# Displays specified audio segment in Jupyter window using IPython.display
def display_clip(wav_path,start_time,end_time):
    track_data = AudioSegment.from_wav(file=wav_path)
    track_data = track_data.set_channels(1)
    bit_depth = track_data.sample_width * 8
    clip_data = track_data[int(1000*start_time):int(1000*end_time)]
    array_type = get_array_type(bit_depth)
    numeric_array = array.array(array_type, clip_data._data)
    display(Audio(numeric_array, rate=track_data.frame_rate))

# Classifies audio at 1-second resolution, plots results if speaker found, 
# and returns speaker ranges as list of 2-tuples.
# Add 1 to 2nd speaker in each 2-tuple for inclusive time span.




def find_speaker(audio_path,classifier_model_path):
    classifier_model_name = classifier_model_path.split('/')[-1]
    classifier_model_type = classifier_model_name.split('_')[0].lower() # assuming model file begins svm_etc
    
    buffer_secs=0
    is_mp3=False
    if audio_path.lower()[-4:] in ['.mp3','.mp4','.wav']:    # Creates a temporary WAV
        is_mp3=True                        # if input is MP3
        random.seed(audio_path)
        wav_path='/var/tmp/'+str(random.random())+'_temp.wav' # Filename for temp WAV is a random float
        subprocess.call(['ffmpeg', '-i', audio_path, '-y', '-ar', '16000', '-ac', '1', '-af', "volume=0.99,highpass=f=150, lowpass=f=5000",  wav_path]) # '-y' option overwrites existing file if present
    else:
        wav_path=audio_path
    print(wav_path)
    output, classesAll, acc, CM = aS.mtFileClassification(wav_path, classifier_model_path, classifier_model_type) #or replace with 'svm' etc. as needed
    output = list(output)
    counter=0
    speaker_secs=[]
    for speaker in output:
        if speaker>0.0:
            speaker_secs.append(counter)
        counter+=1
    speaker_ranges=seconds_list_to_ranges(speaker_secs)
    #if len(speaker_ranges)>0:
    #    print speaker_ranges
    #    print '\n'
    #    pd.Series(output).plot()                      # uncomment for A/V feedback in notebook
    #    plt.show()
    #for pair in speaker_ranges:
    #    print pair
    #    display_clip(wav_path,pair[0],pair[1]+1)
    if is_mp3==True:
        os.remove(wav_path)
    outputfile=audio_path.split('/')[-1][:-4]+"_"+classifier_model_name+".csv"
    with open(outputfile, 'w') as csv_fo:
        speaker_ranges_expanded=[(start,1,end-start+1) for start,end in speaker_ranges]
        csv_writer = csv.writer(csv_fo)
        csv_writer.writerows(speaker_ranges_expanded)
    print(outputfile)
    return speaker_ranges_expanded

In [27]:
yt_path="/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/YouTube_corpus/"+speaker
test_files=[os.path.join(yt_path,item) for item in os.listdir(yt_path) if ".DS_Store" not in item]
test_files

['/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/YouTube_corpus/King, Martin Luther/Dr. Martin Luther King Jr. Interview-Z6hi3t9JFpE.wav',
 '/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/YouTube_corpus/King, Martin Luther/Dr. Martin Luther King Jr. Interview-Z6hi3t9JFpE_svm_King_UBM_900x2s.csv',
 '/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/YouTube_corpus/King, Martin Luther/first interview with Martin Luther King from 1957 part 1--Ll4QmvnGcU.wav',
 '/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/YouTube_corpus/King, Martin Luther/first interview with Martin Luther King from 1957 part 1--Ll4QmvnGcU_svm_King_UBM_900x2s.csv',
 '/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/YouTube_corpus/King, Martin Luther/Long Lost Martin Luther King, Jr. Speech (Complete - Best Audio)-N0Jzqiqwo5A.wav',
 '/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/YouTube_corpus/King, Martin Luther/Long Lost Martin Luther King, Jr. Speech (Complete - Best Audio)-N0Jzqiqwo5A_svm_King_UBM_900x2s.csv',
 '/Volumes

In [28]:
classifier_model_paths=["/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips/"+"svm"+model_id]

#classifier_model_path="/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips/randomforest_Baldwin_UBM_564x2s",\

#classifier_model_path="/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips/svm_Baldwin_UBM_564x2s"

#test_files=[\
#'/Users/mclaugh/Desktop/WGBH_Youtube_tests/Malcolm X - Debate with James Baldwin - September 5, 1963--JIp9_IIV3s.mp4',\
#'/Volumes/McLaughlin-6TB-1/Extended_Corpus/James_Baldwin/cpb-aacip-75-48sbchq4.h264.mp4',\
#'/Users/mclaugh/Desktop/WGBH_Youtube_tests/Julia Child makes an omelet-RThnq3-d6PY.wav',\
#'/Volumes/McLaughlin-6TB-1/Extended_Corpus/James_Baldwin/cpb-aacip-15-719kdkmx.h264.mp4',\
#'/Volumes/McLaughlin-6TB-1/Extended_Corpus/James_Baldwin/cpb-aacip-28-n872v2ct06__PRA_AAPP_BB0838_Baldwin_at_the_Masonic_Temple_.mp3'\
#]

In [29]:
# Example speaker search
import timeit
tic=timeit.default_timer()
#print(timeit.default_timer() - tic)

#audio_path=test_files[2]

for classifier_model_path in classifier_model_paths:
    for audio_path in test_files:
        try: 
            dd=find_speaker(audio_path,classifier_model_path)
            print(timeit.default_timer() - tic)
        except: pass

/var/tmp/0.516010216885_temp.wav
Dr. Martin Luther King Jr. Interview-Z6hi3t9JFpE_svm_King_UBM_4000x2s.csv
118.70905304
/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/YouTube_corpus/King, Martin Luther/Dr. Martin Luther King Jr. Interview-Z6hi3t9JFpE_svm_King_UBM_900x2s.csv
Error in readAudioFile(): Unknown file type!
/var/tmp/0.575053418831_temp.wav
first interview with Martin Luther King from 1957 part 1--Ll4QmvnGcU_svm_King_UBM_4000x2s.csv
138.239279032
/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/YouTube_corpus/King, Martin Luther/first interview with Martin Luther King from 1957 part 1--Ll4QmvnGcU_svm_King_UBM_900x2s.csv
Error in readAudioFile(): Unknown file type!
/var/tmp/0.240233239605_temp.wav
Long Lost Martin Luther King, Jr. Speech (Complete - Best Audio)-N0Jzqiqwo5A_svm_King_UBM_4000x2s.csv
236.56347394
/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/YouTube_corpus/King, Martin Luther/Long Lost Martin Luther King, Jr. Speech (Complete - Best Audio)-N0Jzqiqwo5A_svm_King_UBM

In [30]:
print(timeit.default_timer() - tic)

1956.87235594
